In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
training.head()

In [ ]:
test.head()

In [ ]:
training['train_test']=1
training.head()

In [ ]:
test['train_test']=0
test.head()

In [ ]:
test['Survived']=np.NaN
test.head()

In [ ]:
all_data=pd.concat([training, test])
all_data.head()

In [ ]:
training.info()

In [ ]:
training.columns


In [ ]:
df_num=training[['Age', 'SibSp', 'Parch', 'Fare']]
df_cat=training[['PassengerId', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked' ]]

In [ ]:
training.describe()

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [ ]:
print(df_num.corr())

In [ ]:
pd.pivot_table(training, index='Survived', values=['Age', 'SibSp', 'Parch', 'Fare'])

In [ ]:
pd.pivot_table(training, index='Survived', values=['Age', 'SibSp', 'Parch', 'Fare'], aggfunc='count')

In [ ]:
for i in df_cat.columns:
    plt.bar(df_cat[i], df_cat['Survived'])
    plt.title(i)
    plt.show()

In [ ]:
for i in df_cat.columns:
    sns.barplot(df_cat[i].value_counts().index,df_cat[i].value_counts()).set_title(i)
    plt.show()
    

In [ ]:
print(pd.pivot_table(training, index='Survived', columns='Pclass', aggfunc='count', values='PassengerId'))
print(pd.pivot_table(training, index='Survived', columns='Sex', aggfunc='count', values='PassengerId'))
print(pd.pivot_table(training, index='Survived', columns='Cabin', aggfunc='count', values='PassengerId'))
print(pd.pivot_table(training, index='Survived', columns='Embarked', aggfunc='count', values='PassengerId'))

In [ ]:
training.head()
df_cat.columns

In [ ]:
training['Ticket'].unique

In [ ]:
#Feature Engineering
print(df_cat.Cabin)

In [ ]:
training.Cabin.head(50)

In [ ]:
training['Cabin_mult']=training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
training['Cabin_mult'].value_counts()

In [ ]:
training['Cabin_cat']=training.Cabin.apply(lambda x: str(x)[0])
print(training.Cabin_cat.value_counts())

In [ ]:
pd.pivot_table(training, index='Survived', columns='Cabin_cat', values='PassengerId', aggfunc='count')

In [ ]:
# Cabin D, E high survival rate
#Cabin n low survival rate
training.head()

In [ ]:
#Ticket values
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').
                                                   replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
training['numeric_ticket'].value_counts()

In [ ]:
pd.set_option("max_rows", None)
training['ticket_letters'].value_counts()

In [ ]:
pd.pivot_table(training, index='Survived', columns='numeric_ticket', values='PassengerId', aggfunc='count')

In [ ]:
pd.pivot_table(training, index='Survived', columns='ticket_letters', values='PassengerId', aggfunc='count')

In [ ]:
training.Name.head(50)

In [ ]:
training['Name_info']=training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
training['Name_info'].value_counts()

In [ ]:
training['Name_info'].head(10)

In [ ]:
training['Cabin_cat'].head(10)

#Data processing 
all_data['Cabin_mult']=training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['Cabin_cat']=training.Cabin.apply(lambda x: str(x)[0])
all_data['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
all_data['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').
                                                   replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
all_data.Age=all_data.Age.fillna(training.Age.median())
all_data.Fare=all_data.Fare.fillna(training.Fare.median())
all_data.dropna(subset=['Embarked'], inplace=True)

In [ ]:
#Data processing
all_data['Cabin_mult']=training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['Cabin_cat']=training.Cabin.apply(lambda x: str(x)[0]) 
all_data['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0) 
all_data['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.',''). replace('/','').lower() 
                                                   if len(x.split(' ')[:-1]) >0 else 0) 
all_data['Name_info']=training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
all_data.Age=all_data.Age.fillna(training.Age.median()) 
all_data.Fare=all_data.Fare.fillna(training.Fare.median()) 
all_data.dropna(subset=['Embarked'], inplace=True)

In [ ]:
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_sibsp'].hist()


In [ ]:
all_data['norm_fare'] = np.log(all_data.Fare+1)
all_data['norm_fare'].hist()

In [ ]:
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_sibsp'].hist()
all_data['norm_fare'] = np.log(all_data.Fare+1)
all_data['norm_fare'].hist()

In [ ]:
all_data.columns

In [ ]:
all_data.Pclass=all_data.Pclass.astype(str)
all_dummies = pd.get_dummies(all_data[['Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked',
                                       'Cabin_cat','Cabin_mult','numeric_ticket','Name_info','train_test']])

X_train = all_dummies[all_dummies.train_test == 1].drop(['train_test'], axis =1)
X_test = all_dummies[all_dummies.train_test == 0].drop(['train_test'], axis =1)

y_train = all_data[all_data.train_test==1].Survived
y_train.shape

In [ ]:
y_train.head(10)

In [ ]:
#Scale data
from sklearn.preprocessing import StandardScaler 
scale=StandardScaler()
all_dummies_scaled=all_dummies.copy()
all_dummies_scaled[['Age', 'SibSp', 'Parch', 'norm_fare']]=scale.fit_transform(all_dummies_scaled[['Age', 'SibSp', 'Parch', 'norm_fare']])
all_dummies_scaled.head()

In [ ]:
X_train_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 1].drop(['train_test'], axis =1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 0].drop(['train_test'], axis =1)

y_train = all_data[all_data.train_test==1].Survived

In [ ]:
#Model Building
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [ ]:
gnb = GaussianNB()
cv = cross_val_score(gnb,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state=1)
cv = cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state=1)
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
svc=SVC(probability=True)
cv = cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
svc=SVC(probability=True)
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier(random_sate=1)
cv = cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier(random_sate=1)
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lr',lr),('knn',knn),('rf',rf),('gnb',gnb),('svc',svc),('xgb',xgb)], voting = 'soft') 
cv = cross_val_score(voting_clf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
voting_clf.fit(X_train_scaled, y_train)
y_predict=voting_clf.predict(X_test_scaled).astype(int)
submission={'PassengerId': test.PassengerId, 'Survived': y_predict}
submission=pd.DataFrame(data=submission)
submission.to_csv('submission.csv', index=False)
